In [1]:
import numpy as np
import pandas as pd
import time
import timeit
from datetime import datetime
import os
import glob
import natsort
import shutil
from tqdm.notebook import tqdm
import re
import sys
import matplotlib.pyplot as plt
plt.rcParams['image.cmap'] = 'gray'
import cv2
from PIL import Image
import random
import copy
import warnings
warnings.filterwarnings('ignore')
# import ipynbname
FILENAME = os.getcwd()+'/'+str(__session__).split('/')[-1]

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import torchvision
from torchvision import datasets
from torchvision import transforms
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset, DataLoader 
from monai.losses import TverskyLoss as TverskyLoss
import sys
sys.path.append("..")
from itertools import product  # [ABLATION] 카테시안 곱
Project_Name = 'Crack_Segmentation'

model_dir = 'models'

module_names = ['CrackFormer II','CSNet','DECSNet','LMNet','RSNet']
ablation_space = {
    'use_ms':  [True],
    'use_gate':[True],
}
# ablation_space = {
#     'use_ms':  [True],
#     'use_gate':[True],
# }
model_names = module_names

for module_name in module_names:
    exec(f'from {model_dir}.{module_name} import *')

iterations = [1,5]
train_mode = 'train' # train or inference
output_root = 'output'
# inference시 아래 model root(output)와 실험을 원하는 results csv(전체 실험 파일)을 변수로 입력
if train_mode=='inference':
    past_output_root = f'output' # if train_mode=='inference'
    past_result_csv = 'past.csv' # output vis시 수정 1/2
    past_result_df = pd.read_csv(past_result_csv)
    iteration_dataset_model_tuples = list(zip(past_result_df['Iteration'], past_result_df['Dataset Name'], past_result_df['Model Name']))
if train_mode=='train_again':
    past_result_csv = '241104_past_models_errors.csv'
    past_result_df = pd.read_csv(past_result_csv)
    iteration_dataset_model_tuples = list(zip(past_result_df['Iteration'], past_result_df['Dataset Name'], past_result_df['Model Name']))

# visualization시 true로 만들고 저장원하는 샘플 수를 지정 # output vis시 수정 2/2
SAVE_RESULT = False
SAVE_N = 1500
vis_root = f'TEST_OUTPUTS_{datetime.now().strftime("%y%m%d_%H%M%S")}'
# train시 수행하는 visualization
TRAININGSET_VIS=False
TRAININGSET_VIS_dir = f'TRAININGSET_VIS_{datetime.now().strftime("%y%m%d_%H%M%S")}'
if TRAININGSET_VIS:
    os.makedirs(TRAININGSET_VIS_dir, exist_ok=True)

Dataset_root = '../Total_Datasets/Crack_Segmentation_Dataset'
# Dataset_informs = Dataset_root+'/250507_Crack_Segmentation_Dataset_Inform_v12.csv'
Dataset_informs = Dataset_root+'/250822_Fast_Crack_Segmentation_Dataset_Inform_v3.csv'
# Dataset_informs = Dataset_root+'/241229_Crack_Segmentation_Dataset_Inform_tmp.csv'
df_Dataset_informs = pd.read_csv(Dataset_informs)

# 실험 원하는 데이터셋에 대해서 인덱싱
#df_Dataset_informs = pd.concat([df_Dataset_informs[:19], df_Dataset_informs[-13:], df_Dataset_informs[38:38+20]], ignore_index=True) # without cityscapes
Dataset_Name_list = natsort.natsorted(df_Dataset_informs['Dataset Name'].tolist())
# Dataset_Name_list.remove('L2. Pascal-VOC-2012')
for Dataset_Name in Dataset_Name_list:
    assert os.path.exists(os.path.join(Dataset_root, Dataset_Name)), f"Error: The dataset path '{Dataset_root}/{Dataset_Name}' does not exist."
ratio_combinations = [
    (0.6, 0.2, 0.2)
]

# 하이퍼파라미터 설정
epochs = 100
EARLY_STOP = 100
batch_size = 4
drop_last = True
fill_last_batch = False
devices = [0]

optimizer = 'AdamW'
lr = 1e-3
momentum = 0.9
weight_decay = 1e-4
optim_args = {'optimizer': optimizer, 'lr': lr, 'momentum': momentum, 'weight_decay': weight_decay}

lr_scheduler = 'CosineAnnealingLR'
T_max = epochs
T_0 = epochs
eta_min = 1e-6
lr_scheduler_args = {'lr_scheduler': lr_scheduler, 'T_max': T_max, 'T_0': T_0, 'eta_min': eta_min}

loss_function = 'DiceCELoss'
#loss_function = 'Tversky Focal Loss'
reduction = 'mean'
gamma = 2.0
weight = None
loss_function_args = {'loss_function': loss_function, 'reduction': reduction, 'gamma': gamma, 'weight': weight}

EXCLUDE_BACKGROUND=True

BINARY_SEG = None
THRESHOLD = 0.5

####
augmentation_strs = ["FLIP(HV)=>ROT([90,180,270])"]

ablation_keys = list(ablation_space.keys())
ablation_grid = [dict(zip(ablation_keys, vals)) for vals in product(*ablation_space.values())]

def ablation_to_tag(cfg: dict) -> str:
    """디렉토리/로그 표기를 위한 태그 문자열"""
    return "_".join([f"{k}={str(v)}" for k, v in cfg.items()])

In [2]:
def control_random_seed(seed, pytorch=True):
    random.seed(seed)
    np.random.seed(seed)
    try:
        torch.manual_seed(seed)
        if torch.cuda.is_available()==True:
            torch.cuda.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
    except:
        pass
        torch.backends.cudnn.benchmark = False
def imread_kor ( filePath, mode=cv2.IMREAD_UNCHANGED ) : 
    stream = open( filePath.encode("utf-8") , "rb") 
    bytes = bytearray(stream.read()) 
    numpyArray = np.asarray(bytes, dtype=np.uint8)
    return cv2.imdecode(numpyArray , mode)
def imwrite_kor(filename, img, params=None): 
    try: 
        ext = os.path.splitext(filename)[1] 
        result, n = cv2.imencode(ext, img, params) 
        if result:
            with open(filename, mode='w+b') as f: 
                n.tofile(f) 
                return True
        else: 
            return False 
    except Exception as e: 
        print(e) 
        return False
    
def parse_dimensions(input_str):
    # 정규 표현식을 사용하여 괄호 안의 숫자들을 찾음
    dimensions = re.findall(r'\((\d+),\s*(\d+),\s*(\d+)\)', input_str)
    
    # 괄호가 두 개 이상이면 에러를 출력
    if len(dimensions) > 1:
        return "Error: Multiple or no valid dimensions found."
    
    # 괄호가 하나일 때는 height, width, channels 추출
    if len(dimensions) == 1:
        height, width, _ = map(int, dimensions[0])
        return height, width
    
    # 괄호가 없을 경우 Max Width와 Max Height를 추출
    max_width_match = re.search(r'Max Width:\s*(\d+)', input_str)
    max_height_match = re.search(r'Max Height:\s*(\d+)', input_str)
    
    if max_width_match and max_height_match:
        max_width = int(max_width_match.group(1))
        max_height = int(max_height_match.group(1))
        return max_width, max_height
    
    return "Error: No valid dimensions or Max Width/Height found."
def random_rotation(image, mask, angle_range=(-30, 30)):
    # 지정된 각도 범위 내에서 무작위로 각도 선택
    angle = random.uniform(angle_range[0], angle_range[1])
    # 이미지와 마스크를 동일한 각도로 회전
    image = TF.rotate(image, angle)
    mask = TF.rotate(mask, angle)
    return image, mask
class ImageTransforms:
    def __init__(self, image, mask=None, in_channels=3):
        self.image = image
        self.mask = mask
        self.in_channels = in_channels

    def HWR(self, h, w):
        # 현이미지와 동일한 비율을 유지하며 (h, w) 안에 들어갈 수 있는 최대 크기로 리사이징
        orig_h, orig_w = self.image.shape[:2]
        scale = min(h / orig_h, w / orig_w)
        new_h, new_w = int(orig_h * scale), int(orig_w * scale)
        self.image = cv2.resize(self.image, (new_w, new_h), interpolation=cv2.INTER_LINEAR)
        if self.mask is not None:
            self.mask = cv2.resize(self.mask, (new_w, new_h), interpolation=cv2.INTER_NEAREST)
        return self

    def P(self, h, w):
        # 이미지에 zero-padding을 추가하여 (h, w) 크기로 만듦
        pad_h = max(0, h - self.image.shape[0])
        pad_w = max(0, w - self.image.shape[1])
        top = pad_h // 2
        bottom = pad_h - top
        left = pad_w // 2
        right = pad_w - left
        if self.in_channels == 1:
            value = 0
        else:
            value = [0] * self.in_channels
        self.image = cv2.copyMakeBorder(self.image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=value)
        if self.mask is not None:
            self.mask = cv2.copyMakeBorder(self.mask, top, bottom, left, right, cv2.BORDER_CONSTANT, value=0)
        return self

    def R(self, h, w):
        # 이미지를 (h, w) 크기로 리사이즈
        self.image = cv2.resize(self.image, (w, h), interpolation=cv2.INTER_LINEAR)
        if self.mask is not None:
            self.mask = cv2.resize(self.mask, (w, h), interpolation=cv2.INTER_NEAREST)
        return self

    def SQP(self):
        # 정사각형으로 만들기 위해 짧은 축 기준으로 zero-padding
        h, w = self.image.shape[:2]
        size = max(h, w)
        self.P(size, size)
        return self
    def C(self, h, w):
        # 이미지를 (h, w) 크기로 랜덤 크롭핑
        img_h, img_w = self.image.shape[:2]
        if img_h < h or img_w < w:
            raise ValueError("크롭 크기가 이미지 크기보다 큽니다. 먼저 패딩 또는 리사이징을 수행하세요.")
        
        top = np.random.randint(0, img_h - h + 1)
        left = np.random.randint(0, img_w - w + 1)
        
        self.image = self.image[top:top + h, left:left + w]
        if self.mask is not None:
            self.mask = self.mask[top:top + h, left:left + w]
        return self
    def get_image_and_mask(self):
        return self.image, self.mask

    def get_image_and_mask(self):
        return self.image, self.mask

class ImagesDataset(Dataset):
    def __init__(self, image_path_list, target_path_list, Height, Width, augmentation_str=None, transform_str=None):
        self.image_path_list = image_path_list
        self.target_path_list = target_path_list
        self.transform = transforms.Compose([
            transforms.ToTensor(),
        ])
        self.Height = Height
        self.Width = Width
        self.transform_str = transform_str
        self.augmentation_str = augmentation_str
    def __len__(self):
        return len(self.image_path_list)

    def apply_augmentation(self, image_pil, mask_pil):
        operations = self.augmentation_str.split("=>")
        for op in operations:
            op = op.strip()
            match_flip = re.match(r"FLIP\((H|V|HV)\)", op)
            match_rot_list = re.search(r"ROT\(\[([\d,]+)\]\)", op)
            match_rot_range = re.match(r"ROT\((-?\d+(?:\.\d+)?),\s*(-?\d+(?:\.\d+)?)\)", op)
            match_zoom = re.match(r"ZOOM\((\d+(?:\.\d+)?),\s*(\d+(?:\.\d+)?)\)", op)
            angle = 0
            flip = 'NoFlip'
            resize_scale = 1 
            if match_flip:
                direction = match_flip.group(1)
                if direction == "H":
                    if random.random() < 0.5:
                        image_pil = TF.hflip(image_pil)
                        mask_pil = TF.hflip(mask_pil)
                elif direction == "V":
                    if random.random() < 0.5:
                        image_pil = TF.vflip(image_pil)
                        mask_pil = TF.vflip(mask_pil)
                elif direction == "HV":
                    if random.random() < 0.5:
                        image_pil = TF.hflip(image_pil)
                        mask_pil = TF.hflip(mask_pil)
                    if random.random() < 0.5:
                        image_pil = TF.vflip(image_pil)
                        mask_pil = TF.vflip(mask_pil)
            elif match_rot_list:
                angles = [int(num) for num in match_rot_list.group(1).split(",")]
                angle = random.choice(angles)
                image_pil = TF.rotate(image_pil, angle)
                mask_pil = TF.rotate(mask_pil, angle, interpolation=Image.NEAREST)
            elif match_rot_range:
                angle_min = float(match_rot_range.group(1))
                angle_max = float(match_rot_range.group(2))
                angle = random.uniform(angle_min, angle_max)
                image_pil = TF.rotate(image_pil, angle)
                mask_pil = TF.rotate(mask_pil, angle, interpolation=Image.NEAREST)
            elif match_zoom:
                zoom_min = float(match_zoom.group(1))
                zoom_max = float(match_zoom.group(2))
                scale = random.uniform(zoom_min, zoom_max)
                w, h = image_pil.size
                new_w, new_h = int(w * scale), int(h * scale)
                image_pil = TF.resize(image_pil, (new_h, new_w))
                mask_pil = TF.resize(mask_pil, (new_h, new_w), interpolation=Image.NEAREST)
                image_pil = TF.center_crop(image_pil, (h, w))
                mask_pil = TF.center_crop(mask_pil, (h, w))
                resize_scale = scale
        return image_pil, mask_pil, (flip, angle, resize_scale)
        
    def parse_transform_str(self, image, mask):
        in_channels = 1 if len(image.shape) == 2 else image.shape[2]
        transform = ImageTransforms(image, mask, in_channels)
        if self.transform_str:
            operations = self.transform_str.split("=>")
            for op in operations:
                op = op.strip()
                match_hwr = re.match(r"(HWR|P|R|C)\((\d+),\s*(\d+)\)", op)
                match_sqp = re.match(r"SQP", op)
                if match_hwr:
                    func = match_hwr.group(1)
                    h = int(match_hwr.group(2))
                    w = int(match_hwr.group(3))
                    if func == "HWR":
                        transform.HWR(h, w)
                    elif func == "P":
                        transform.P(h, w)
                    elif func == "R":
                        transform.R(h, w)
                    elif func == "C":
                        transform.C(h, w)
                elif match_sqp:
                    transform.SQP()
        return transform.get_image_and_mask()

    def __getitem__(self, idx):
        image_path = self.image_path_list[idx]
        mask_path = self.target_path_list[idx]
        image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        if image is None:
            raise ValueError(f"Image not found at {image_path}")

        mask = np.load(mask_path)
        mask = mask.astype(np.uint8)  # Ensure mask is in the correct format

        # 데이터 증강 적용
        flip='NoFlip'; angle=0; resize_scale = 1.0;
        if self.augmentation_str and (random.random() > 0.5):
            if isinstance(image, np.ndarray):
                image_pil = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB) if len(image.shape) == 3 else image)
                mask_pil = Image.fromarray(mask)
            else:
                image_pil = image
                mask_pil = mask
            image_pil, mask_pil, aug_params = self.apply_augmentation(image_pil, mask_pil)
            flip, angle, resize_scale = aug_params

            image = np.array(image_pil)
            mask = np.array(mask_pil)
            if len(image.shape) == 3:
                image = image[..., ::-1]  # Convert RGB back to BGR for OpenCV

        # Apply custom transformations
        transformed_image, transformed_mask = self.parse_transform_str(image, mask)
        
        if TRAININGSET_VIS==True and self.augmentation_str:
            TRAININGSET_DATA_VIS_dir = TRAININGSET_VIS_dir+'/'+Dataset_Name
            os.makedirs(TRAININGSET_DATA_VIS_dir, exist_ok=True)
            shutil.copy(image_path, os.path.join(TRAININGSET_DATA_VIS_dir, f'{Dataset_Name}_{idx}_{os.path.basename(image_path)}_1_image_original.png'))
            cv2.imwrite(os.path.join(TRAININGSET_DATA_VIS_dir, f'{Dataset_Name}_{idx}_{os.path.basename(image_path)}_3_mask_original.png'), colorize_mask(np.load(mask_path)))

            # 변환된 이미지와 마스크 시각화 저장
            # 강조: 변환된 이미지를 시각화 목적으로 저장
            cv2.imwrite(os.path.join(TRAININGSET_DATA_VIS_dir, f'{Dataset_Name}_{idx}_{os.path.basename(image_path)}_2_image_RZ{resize_scale:.2f}_{flip}_RT{int(angle)}.png'), transformed_image)
            cv2.imwrite(os.path.join(TRAININGSET_DATA_VIS_dir, f'{Dataset_Name}_{idx}_{os.path.basename(image_path)}_4_mask_RZ{resize_scale:.2f}_{flip}_RT{int(angle)}.png'), colorize_mask(transformed_mask))

        image = self.transform(transformed_image.copy()).float()
        mask = torch.tensor(transformed_mask).long().unsqueeze(0)
        # print('Image-Mask verification',image.shape, image.mean(), image.max(),image.min(), np.unique(mask.numpy()), end=' ')
        return image, mask, image_path
        
class SegDataLoader(DataLoader):
    def __init__(self, dataset, batch_size=1, shuffle=False, sampler=None,
                 batch_sampler=None, num_workers=0, collate_fn=None,
                 pin_memory=False, drop_last=False, fill_last_batch=False):
        self.fill_last_batch = fill_last_batch
        self.dataset = dataset
        super().__init__(dataset, batch_size, shuffle, sampler,
                         batch_sampler, num_workers, collate_fn,
                         pin_memory, drop_last)

    def __iter__(self):
        batch_iter = super().__iter__()
        for batch in batch_iter:
            if self.fill_last_batch and len(batch[0]) < self.batch_size:
                additional_samples_needed = self.batch_size - len(batch[0])
                additional_indices = random.choices(range(len(self.dataset)), k=additional_samples_needed)
                additional_samples = [self.dataset[idx] for idx in additional_indices]

                if isinstance(batch, (list, tuple)):
                    batch = list(batch)
                    for i in range(len(batch) - 1):  # Process tensor elements (image, mask)
                        batch[i] = torch.cat([batch[i], torch.stack([sample[i] for sample in additional_samples])])
                    batch[-1] = batch[-1] + [sample[-1] for sample in additional_samples]  # Process string elements (image paths)
                    batch = tuple(batch)
                else:
                    batch = torch.cat([batch, torch.stack(additional_samples)])
            yield batch
# 클래스별 고유 색상 지정 (클래스 0은 배경, 나머지는 각 클래스에 고유 색상)
class_colors = {
    0: [0, 0, 0],         # 클래스 0: 검정색 (배경)
    1: [255, 0, 0],       # 클래스 1: 빨강
    2: [0, 255, 0],       # 클래스 2: 초록
    3: [0, 0, 255],       # 클래스 3: 파랑
    4: [255, 255, 0],     # 클래스 4: 노랑
    5: [255, 0, 255],     # 클래스 5: 분홍
    6: [0, 255, 255],     # 클래스 6: 하늘색
    7: [128, 0, 128],     # 클래스 7: 보라색
    8: [128, 128, 0],     # 클래스 8: 올리브색
    9: [0, 128, 128],     # 클래스 9: 청록색
    10: [128, 128, 128],  # 클래스 10: 회색
    11: [192, 0, 0],      # 클래스 11: 진한 빨강
    12: [0, 192, 0],      # 클래스 12: 진한 초록
    13: [0, 0, 192],      # 클래스 13: 진한 파랑
    14: [192, 192, 0],    # 클래스 14: 연한 노랑
    15: [192, 0, 192],    # 클래스 15: 연한 분홍
    16: [0, 192, 192],    # 클래스 16: 연한 하늘색
    17: [128, 64, 0],     # 클래스 17: 갈색
    18: [64, 128, 0],     # 클래스 18: 연두색
    19: [0, 64, 128],     # 클래스 19: 어두운 청록색
    20: [255, 128, 0],    # 클래스 20: 주황색
    21: [128, 0, 255],    # 클래스 21: 보라색 (밝은)
    22: [0, 128, 255],    # 클래스 22: 밝은 파랑
    23: [255, 128, 128],  # 클래스 23: 연한 빨강
    24: [128, 255, 128],  # 클래스 24: 연한 초록
    25: [128, 128, 255],  # 클래스 25: 연한 파랑
    26: [255, 255, 128],  # 클래스 26: 연한 노랑
    27: [255, 0, 128],    # 클래스 27: 핫핑크
    28: [128, 255, 0],    # 클래스 28: 형광 초록
    29: [0, 255, 128],    # 클래스 29: 밝은 청록색
}

    
def colorize_mask(mask):
    # 마스크에 색상 입히기
    color_mask = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
    for cls, color in class_colors.items():
        color_mask[mask == cls] = color
    return color_mask
    
class DiceCELoss:
    """
    Dice Loss와 CE Loss의 결합 손실 클래스
    """
    def __init__(self, weight=0.5, epsilon=1e-6, mode='multiclass'):
        """
        Args:
            weight (float): Dice Loss와 BCE Loss 사이의 가중치 (0~1 사이의 값)
            epsilon (float): 0으로 나누는 것을 방지하기 위한 작은 값
            mode (str): 'binary' 또는 'multiclass'로 손실 계산 모드를 설정
        """
        self.weight = weight
        self.epsilon = epsilon
        self.mode = mode
    
    def __call__(self, pred, target):
        """
        결합 손실 계산 함수
        
        Args:
            pred (torch.Tensor): 예측된 확률맵, 
                - binary segmentation: shape이 (batchsize, 1, H, W)
                - multiclass segmentation: shape이 (batchsize, num_classes, H, W)
            target (torch.Tensor): 정답 마스크, shape이 (batchsize, 1, H, W)
            
        Returns:
            torch.Tensor: 계산된 결합 손실 값
        """
        if self.mode == 'binary':
            # Binary Dice Loss 계산
            pred = pred.squeeze(1)  # shape: (batchsize, H, W)
            target = target.squeeze(1).float()
            intersection = torch.sum(pred * target, dim=(1, 2))
            union = torch.sum(pred, dim=(1, 2)) + torch.sum(target, dim=(1, 2))
            dice = (2 * intersection + self.epsilon) / (union + self.epsilon)
            dice_loss = 1 - dice.mean()
            
            # BCE Loss 계산
            ce_loss = F.binary_cross_entropy(pred, target)
        
        elif self.mode == 'multiclass':
            # Multiclass Dice Loss 계산
            batchsize, num_classes, H, W = pred.shape
            target = target.squeeze(1)
            target_one_hot = F.one_hot(target, num_classes=num_classes).squeeze(1).permute(0, 3, 1, 2).float()
            intersection = torch.sum(pred * target_one_hot, dim=(2, 3))
            union = torch.sum(pred, dim=(2, 3)) + torch.sum(target_one_hot, dim=(2, 3))
            dice = (2 * intersection + self.epsilon) / (union + self.epsilon)
            dice_loss = 1 - dice.mean()
            
            # Cross Entropy Loss 계산
            ce_loss = F.cross_entropy(pred, target)
        else:
            raise ValueError("mode should be 'binary' or 'multiclass'")
        
        # 결합 손실 계산
        combined_loss = self.weight * dice_loss + (1 - self.weight) * ce_loss
        
        return combined_loss

class DiceCoefficient:
    def __init__(self, num_classes=None):
        self.num_classes = num_classes

    def __call__(self, y_pred, y_true):
        y_true_one_hot = np.eye(self.num_classes)[y_true]
        y_pred_one_hot = np.eye(self.num_classes)[y_pred]
        intersection = np.sum(y_true_one_hot * y_pred_one_hot, axis=(1, 2))
        union = np.sum(y_true_one_hot, axis=(1, 2)) + np.sum(y_pred_one_hot, axis=(1, 2))
        dice = (2. * intersection) / (union + 1e-6)
        return dice

class IoU:
    def __init__(self, num_classes=None):
        self.num_classes = num_classes

    def __call__(self, y_pred, y_true):
        y_true_one_hot = np.eye(self.num_classes)[y_true]
        y_pred_one_hot = np.eye(self.num_classes)[y_pred]
        intersection = np.sum(y_true_one_hot* y_pred_one_hot, axis=(1, 2))
        union = np.sum(y_true_one_hot, axis=(1, 2)) + np.sum(y_pred_one_hot, axis=(1, 2)) - intersection
        iou = intersection / (union + 1e-6)
        return iou

class Precision:
    def __init__(self, num_classes=None):
        self.num_classes = num_classes

    def __call__(self, y_pred, y_true):
        y_true_one_hot = np.eye(self.num_classes)[y_true]
        y_pred_one_hot = np.eye(self.num_classes)[y_pred]
        tp = np.sum(y_true_one_hot* y_pred_one_hot, axis=(1, 2))
        fp = np.sum((y_pred_one_hot == 1) & (y_true_one_hot== 0), axis=(1, 2))
        precision = tp / (tp + fp + 1e-6)
        return precision

class Recall:
    def __init__(self,  num_classes=None):
        self.num_classes = num_classes

    def __call__(self, y_pred, y_true):
        y_true_one_hot = np.eye(self.num_classes)[y_true]
        y_pred_one_hot = np.eye(self.num_classes)[y_pred]
        tp = np.sum(y_true_one_hot* y_pred_one_hot, axis=(1, 2))
        fn = np.sum((y_true_one_hot == 1) & (y_pred_one_hot == 0), axis=(1, 2))
        recall = tp / (tp + fn + 1e-6)
        return recall
    
def check_class_presence(batch_masks, num_classes):
    """
    각 샘플이 각 클래스에 대해 positive를 갖는지 체크하는 함수 (binary, multi class 동일)

    Args:
        batch_masks (numpy.ndarray): shape이 (batchsize, H, W)인 멀티 클래스 GT 마스크 파일
        num_classes (int): 클래스의 수 (0부터 num_classes-1까지의 레이블을 가짐)

    Returns:
        numpy.ndarray: shape이 (batchsize, num_classes)인 Boolean 배열로,
                       각 샘플이 각 클래스에 대해 positive를 가지면 True, 그렇지 않으면 False
    """
    batchsize = batch_masks.shape[0]
    presence_matrix = np.zeros((batchsize, num_classes), dtype=bool)
    for i in range(batchsize):
        for c in range(num_classes):
            presence_matrix[i, c] = np.any(batch_masks[i] == c)

    return presence_matrix

    
def train(train_loader, epoch, model, criterion, optimizer, device, activation ):
    model.train()
    train_losses=AverageMeter()

    # for i, (input, target, _) in enumerate(tqdm(train_loader, desc="Training", unit="batch", leave=False)):
    for i, (input, target, _) in enumerate(train_loader):
        # print( f'{i+1}/{len(train_loader)} {datetime.now().strftime("%y%m%d_%H%M%S")}', end=' ')
        input = input.to(device)
        target = target.to(device)

        output = activation(model(input)) 

        loss = criterion(output,target).float()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_losses.update(loss.detach().cpu().numpy(),input.shape[0])
    Train_Loss=np.round(train_losses.avg,6)
    return Train_Loss
def validate(validation_loader, model, criterion, num_classes, device, activation, model_path=False):
    if model_path:
        model.load_state_dict(torch.load(model_path))
    model.eval()
    iou_calculator = IoU(num_classes)
    
    total_loss = 0.0
    total_samples = 0
    
    columns = ['image_path'] + [f'class_{i}' for i in range(num_classes)]
    metrics_df = pd.DataFrame(columns=['image_path'] + [f'{metric}_class_{i}' for metric in ['iou'] for i in range(num_classes)])

    # for i, (input, target, image_path) in enumerate(tqdm(validation_loader, desc="Valiation", unit="batch", leave=False)):
    for i, (input, target, image_path) in enumerate(validation_loader):
        input = input.to(device)
        target = target.to(device)
        batch_size = input.size(0)
        with torch.no_grad():
            output = activation(model(input))
            loss = criterion(output, target).float()
            
        # 예측 결과 및 타겟을 numpy 배열로 변환
        output_np = np.squeeze(np.where(output.cpu().numpy() > THRESHOLD, 1, 0), axis=1) if BINARY_SEG else torch.argmax(output, dim=1).cpu().numpy()
        target_np = target.cpu().numpy()
        target_np = np.squeeze(target_np, axis=1)
        
        # 클래스 존재 여부 확인
        presence_matrix = check_class_presence(target_np, num_classes=num_classes)
        
        # IoU 계산 (클래스가 존재하는 경우에만)
        iou_values = iou_calculator(output_np, target_np)
        iou_values *= presence_matrix

        # presence_matrix가 True인 경우에만 데이터프레임에 IoU 값 추가
        metrics_row = {'image_path': image_path}
        for j in range(num_classes):
            if presence_matrix[:, j].any():
                metrics_row[f'iou_class_{j}'] = iou_values[:, j].mean() if presence_matrix[:, j].any() else np.nan
        metrics_df = metrics_df.append(metrics_row, ignore_index=True)
        
        # 손실 업데이트
        total_loss += loss.item() * batch_size
        total_samples += batch_size

    mean_loss = total_loss / total_samples
    return metrics_df, mean_loss 

def test(test_loader, model, criterion, device, num_classes, activation, model_path=False):
    if model_path:
        model.load_state_dict(torch.load(model_path))
    model.eval()
    
    dice_calculator = DiceCoefficient(num_classes)
    iou_calculator = IoU(num_classes)
    precision_calculator = Precision(num_classes) 
    recall_calculator = Recall(num_classes) 
    
    total_loss = 0.0
    total_samples = 0
    if SAVE_RESULT:
        save_n = 0
        save_bool = False
    columns = ['image_path'] + [f'class_{i}' for i in range(num_classes)]
    metrics_df = pd.DataFrame(columns=['image_path'] + [f'{metric}_class_{i}' for metric in ['iou', 'dice', 'precision', 'recall'] for i in range(num_classes)])

    for i, (input, target, image_path) in enumerate(tqdm(test_loader, desc="Test", unit="batch")):
        input = input.to(device)
        target = target.to(device)
        with torch.no_grad():
            output = activation(model(input))
            loss = criterion(output, target).float()

        # 예측 결과 및 타겟을 numpy 배열로 변환
        output_np = np.squeeze(np.where(output.cpu().numpy() > THRESHOLD, 1, 0), axis=1) if BINARY_SEG else torch.argmax(output, dim=1).cpu().numpy()
        target_np = target.cpu().numpy()
        target_np = np.squeeze(target_np, axis=1)

        # 클래스 존재 여부 확인
        presence_matrix = check_class_presence(target_np, num_classes=num_classes)

        # 메트릭 계산 (클래스가 존재하는 경우에만)
        iou_values = iou_calculator(output_np, target_np)
        dice_values = dice_calculator(output_np, target_np)
        precision_values = precision_calculator(output_np, target_np)
        recall_values = recall_calculator(output_np, target_np)

        # presence_matrix가 True인 경우에만 데이터프레임에 메트릭 값 추가
        batch_metrics = []
        for b in range(input.shape[0]):
            metrics_row = {'image_path': os.path.basename(image_path[b])}
            for j in range(num_classes):
                metrics_row[f'iou_class_{j}'] = iou_values[b, j] if presence_matrix[b, j] else np.nan
                metrics_row[f'dice_class_{j}'] = dice_values[b, j] if presence_matrix[b, j] else np.nan
                metrics_row[f'precision_class_{j}'] = precision_values[b, j] if presence_matrix[b, j] else np.nan
                metrics_row[f'recall_class_{j}'] = recall_values[b, j] if presence_matrix[b, j] else np.nan
            batch_metrics.append(metrics_row)

        metrics_df = pd.concat([metrics_df, pd.DataFrame(batch_metrics)], ignore_index=True)

        # 손실 업데이트
        total_loss += loss.item() * input.shape[0]
        total_samples += input.shape[0]

        if SAVE_RESULT:
            os.makedirs(vis_root, exist_ok=True)
            for j, (out, tar, path) in enumerate(zip(output_np, target_np, image_path)):
                save_n+=1
                img_name = os.path.basename(path)
                np.save(vis_root+f'/{Dataset_Name}_{save_n}_{img_name}_{model_name}_output_Iter_{iteration}.npy', out)
                np.save(vis_root+f'/{Dataset_Name}_{save_n}_{img_name}_{model_name}_target_Iter_{iteration}.npy', tar)
                if SAVE_RESULT and save_n>=SAVE_N:
                    save_bool=True
                    break
        if SAVE_RESULT and save_bool:
            break
    mean_loss = total_loss / total_samples
    return metrics_df, mean_loss
def aggregate_measures(metrics_df, exclude_background=True, metric_types=['iou', 'dice', 'precision', 'recall']):
    # Calculate classwise_metrics: average across all image paths for each class and metric
    classwise_metrics = metrics_df.iloc[:, 1:].mean(skipna=True)  # Ignoring the 'image_path' column

    # Calculate samplewise_metrics: average across all classes for each sample (ignoring NaN values)
    # Grouping the metrics into IoU, Dice, Precision, and Recall
    samplewise_metrics_dict = {}

    for metric in metric_types:
        metric_columns = [col for col in metrics_df.columns if metric in col and (not exclude_background or not col.endswith('_0'))]
        samplewise_metrics_dict[f'mean_{metric}'] = metrics_df[metric_columns].mean(axis=1, skipna=True)

    # Create DataFrames for both results with the appropriate column names
    classwise_metrics_df = classwise_metrics.to_frame().T  # Convert to DataFrame with a single row

    # For samplewise_metrics, create a DataFrame with 'image_path' and all averaged metrics
    samplewise_metrics_df = metrics_df[['image_path']].copy()  # Keep 'image_path' column

    for key, value in samplewise_metrics_dict.items():
        samplewise_metrics_df[key] = value
    overall_metrics = {}

    for metric in metric_types:
        metric_columns = [col for col in classwise_metrics_df.columns if metric in col and (not exclude_background or not col.endswith('_0'))]
        overall_metrics[metric] = classwise_metrics_df[metric_columns].mean(axis=1, skipna=True).values[0]

    overall_metrics_df = pd.DataFrame(overall_metrics, index=[0])
    
    return overall_metrics_df, classwise_metrics_df, samplewise_metrics_df
def str_to_class(classname):
    return getattr(sys.modules[__name__], classname)

def copy_sourcefile(output_dir, src_dir = 'src' ):    
    import os 
    import shutil
    import glob 
    source_dir = os.path.join(output_dir, src_dir)

    os.makedirs(source_dir, exist_ok=True)
    org_files1 = os.path.join('./', '*.py' )
    org_files2 = os.path.join('./', '*.sh' )
    org_files3 = os.path.join('./', '*.ipynb' )
    org_files4 = os.path.join('./', '*.txt' )
    org_files5 = os.path.join('./', '*.json' )    
    files =[]
    files = glob.glob(org_files1 )
    files += glob.glob(org_files2  )
    files += glob.glob(org_files3  )
    files += glob.glob(org_files4  ) 
    files += glob.glob(org_files5  )     

    # print("COPY source to output/source dir ", files)
    tgt_files = os.path.join( source_dir, '.' )
    for i, file in enumerate(files):
        shutil.copy(file, tgt_files)
class LossSaver(object):
    def __init__(self):
        self.train_losses = []
        self.val_losses = []
    def reset(self):
        self.train_losses = []
        self.val_losses = []
    def update(self, train_loss, val_loss):
        self.train_losses.append(train_loss)
        self.val_losses.append(val_loss)
    def return_list(self):
        return self.train_losses, self.val_losses
    def save_as_csv(self, csv_file):
        df = pd.DataFrame({'Train Losses': self.train_losses, 'Validation Losses': self.val_losses})
        df.index = [f"{i+1} Epoch" for i in df.index]
        df.to_csv(csv_file, index=True)
class AverageMeter (object):
    def __init__(self):
        self.reset ()
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count        
def create_dataset_lists(Dataset_dir, iteration, data_split_csv):
    # CSV 파일을 읽어옴
    df = pd.read_csv(data_split_csv)
    Originals_dir = Dataset_dir+'/Originals'
    Masks_dir = Dataset_dir+'/Masks'
    
    # 주어진 iteration에 맞는 split 선택
    split_name = f'split{str(iteration).zfill(2)}'  # iteration 번호를 01, 02, ... 형식으로 맞춤
    
    # 선택된 split에 해당하는 데이터만 필터링
    train_data = df[(df['split'] == split_name) & (df['data_type'] == 'training')]
    validation_data = df[(df['split'] == split_name) & (df['data_type'] == 'validation')]
    test_data = df[(df['split'] == split_name) & (df['data_type'] == 'test')]
    
    # 데이터 경로에 Dataset_dir 추가하여 절대 경로 생성
    train_image_path_list = [os.path.join(Originals_dir, path) for path in train_data['image'].tolist()]
    train_target_path_list = [os.path.join(Masks_dir, path) for path in train_data['mask'].tolist()]
    
    validation_image_path_list = [os.path.join(Originals_dir, path) for path in validation_data['image'].tolist()]
    validation_target_path_list = [os.path.join(Masks_dir, path) for path in validation_data['mask'].tolist()]
    
    test_image_path_list = [os.path.join(Originals_dir, path) for path in test_data['image'].tolist()]
    test_target_path_list = [os.path.join(Masks_dir, path) for path in test_data['mask'].tolist()]
    
    
    # 결과 반환
    return (
        train_image_path_list, train_target_path_list, 
        validation_image_path_list, validation_target_path_list, 
        test_image_path_list, test_target_path_list
    )

def Do_Experiment(iteration, model_name, model, train_loader, validation_loader, test_loader, Optimizer, lr,  number_of_classes, epochs, Metrics,df,device, transform, train_mode):
    start = timeit.default_timer()
    if train_mode=='train' or train_mode=='train_again':
        train_bool=True
        test_bool=True
    elif train_mode=='inference':
        past_result_df = pd.read_csv(past_result_csv)
        corresponding_df = past_result_df[(past_result_df['Model Name']==model_name)&
        (past_result_df['Iteration']==iteration)&
        (past_result_df['Dataset Name']==Dataset_Name)]
        Train_date = corresponding_df['Train Time'].item()
        ex_time = corresponding_df['Experiment Time'].item()
        train_bool=False
        test_bool=True
        model_path=past_output_root+f'/output_{ex_time}/{model_name}_{Dataset_Name}_Split_{int(ratio_combination[0]*100)}_{int(ratio_combination[1]*100)}_{int(ratio_combination[2]*100)}_Iter_{iteration}/{Train_date}_{model_name}_{Dataset_Name}_Split_{int(ratio_combination[0]*100)}_{int(ratio_combination[1]*100)}_{int(ratio_combination[2]*100)}_Iter_{iteration}.pt'
    if loss_function == 'Tversky Focal Loss':
        criterion=TverskyLoss()
    elif loss_function == 'DiceCELoss':
        criterion=DiceCELoss(mode='binary') if BINARY_SEG else DiceCELoss(mode='multiclass') 
    if Optimizer=='Adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    elif Optimizer == 'SGD':
        momentum = 0.9
        weight_decay = 1e-4
        optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum ,weight_decay=weight_decay)
    elif Optimizer =='AdamW':
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    if lr_scheduler_args['lr_scheduler'] == 'CosineAnnealingLR':
        lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = lr_scheduler_args['T_max'], eta_min = lr_scheduler_args['eta_min'])
    activation = nn.Sigmoid() if BINARY_SEG else nn.Softmax(1)
    os.makedirs(output_dir, exist_ok = True)
    control_random_seed(seed)
    try:
        if train_bool:
            now = datetime.now()
            Train_date=now.strftime("%y%m%d_%H%M%S")
            print('Training Start Time:',Train_date)
            model_path=f'{output_dir}/{Train_date}_{model_name}_{Dataset_Name}_Split_{int(ratio_combination[0]*100)}_{int(ratio_combination[1]*100)}_{int(ratio_combination[2]*100)}_Iter_{iteration}.pt'
            best=9999
            best_epoch=1
            Early_Stop=0
            loss_saver = LossSaver()
            train_start_time = timeit.default_timer()
            for epoch in range(1, epochs+1):
                Train_Loss = train(train_loader, epoch, model, criterion, optimizer, device, activation)
                lr_scheduler.step()
                metrics_df, Val_Loss  = validate(validation_loader, model, criterion, number_of_classes, device, activation)
                
                overall_metrics_df, classwise_metrics_df, samplewise_metrics_df = aggregate_measures(metrics_df, exclude_background, metric_types = ['iou'])
                
                Val_IoU = overall_metrics_df['iou'].item(); 
    
                date = datetime.now().strftime("%y%m%d_%H%M%S")
                print(f"{epoch}EP({date}): T_Loss: {Train_Loss:.6f} V_Loss: {Val_Loss:.6f} IoU: {Val_IoU:.4f}", end=' ')
                
                loss_saver.update(Train_Loss, Val_Loss)
                loss_saver.save_as_csv(f'{output_dir}/Losses_{Train_date}.csv')
                if Val_Loss<best:
                    Early_Stop = 0
                    torch.save(model.state_dict(), model_path)
                    best_epoch = epoch
                    best = Val_Loss
                    print(f'Best Epoch: {best_epoch} Loss: {Val_Loss:.6f}')
                else:
                    print('')
                    Early_Stop+=1
                if Early_Stop>=EARLY_STOP:
                    break
            train_stop_time = timeit.default_timer()
        if test_bool:
            now = datetime.now()
            date=now.strftime("%y%m%d_%H%M%S")
            print('Test Start Time:',date)
            metrics_df, Test_Loss = test(test_loader, model, criterion, device, number_of_classes, activation, model_path=model_path)
            overall_metrics_df, classwise_metrics_df, samplewise_metrics_df = aggregate_measures(metrics_df, exclude_background, metric_types = ['iou', 'dice', 'precision', 'recall'])
            
            metrics_df.to_csv(f'{output_dir}/Test_sample_class_wise_{model_name}_{Dataset_Name}_Iter_{iteration}_{Train_date}.csv', index=False, header=True, encoding="cp949")
            
            # 각 클래스 평균 계산하여 CSV 저장
            classwise_metrics_df.to_csv(f'{output_dir}/Test_class_wise_{model_name}_Iter_{Dataset_Name}_{iteration}_{Train_date}.csv', index=False, header=True, encoding="cp949")
            # 샘플-클래스 레벨 eval 저장
            samplewise_metrics_df.to_csv(f'{output_dir}/Test_sample_wise_{model_name}_{Dataset_Name}_Iter_{iteration}_{Train_date}.csv', index=False, header=True, encoding="cp949")
                    
            iou = overall_metrics_df['iou'].item(); dice = overall_metrics_df['dice'].item(); 
            precision = overall_metrics_df['precision'].item(); recall = overall_metrics_df['recall'].item();
    
            date = datetime.now().strftime("%y%m%d_%H%M%S")
            if train_mode == 'train' or train_mode == 'train_again' :
                print('Best Epoch:', best_epoch)
    
            print(f"Test({date}): Loss: {Test_Loss:.6f} IoU: {iou:.4f} Dice: {dice:.4f} Precision: {precision:.4f} Recall: {recall:.4f}")

            if train_mode == 'train' or train_mode == 'train_again' :
                stop = timeit.default_timer(); m, s = divmod((train_stop_time - train_start_time)/epoch, 60); h, m = divmod(m, 60); Time_per_Epoch = "%02d:%02d:%02d" % (h, m, s)
                m, s = divmod(stop - start, 60); h, m = divmod(m, 60); Time = "%02d:%02d:%02d" % (h, m, s)
            else:
                Time_per_Epoch = best_epoch = best = Time = 'infer mode'
            
            total_params = sum(p.numel() for p in model.parameters()); total_params = format(total_params , ',')
            
            Performances = [Experiments_Time, Train_date, iteration, Dataset_Name, 
                            f"'{int(ratio_combination[0]*100)}:{int(ratio_combination[1]*100)}:{int(ratio_combination[2]*100)}",
                            model_name, best, Test_Loss, iou, dice, precision, recall, total_params, Time, best_epoch, Time_per_Epoch,
                            loss_function, lr, batch_size, epochs, transform_str, augmentation_str, 
                            f'{len(train_image_path_list)}/{len(validation_image_path_list)}/{len(test_image_path_list)}', FILENAME]
            
            # =========================== [SAFE-ROW-APPEND] ===========================
            # 1) Performances가 채우는 "기본 컬럼 24개"를 명시적으로 정의
            _base_cols = [
                'Experiment Time','Train Time','Iteration','Dataset Name','Data Split','Model Name',
                'Val Loss','Test Loss','IoU','Dice','Precision','Recall','Total Params',
                'Train-Predction Time','Best Epoch','Time per Epoch',
                'Loss Function','LR','Batch size','#Epochs','Preprocessing','Augmentation',
                'Sample Size','DIR'
            ]
            # 2) 기본 컬럼 → 값 매핑
            _row = dict(zip(_base_cols, Performances))
            
            # 3) df.columns에 존재하지만 기본 컬럼에 없는 컬럼들(대개 ablation 변수들)을 채움
            _extra_cols = [c for c in df.columns if c not in _base_cols]
            for c in _extra_cols:
                # ab_cfg가 있으면 그 값, 없으면 None
                _row[c] = (ab_cfg.get(c, None) if 'ab_cfg' in locals() else None)
            
            # 4) df.columns 순서에 맞춰 1행 DataFrame 생성 후 concat
            _row_df = pd.DataFrame([[ _row.get(c, None) for c in df.columns ]], columns=df.columns)
            df = pd.concat([df, _row_df], ignore_index=True)
            # ================================================================================
    except torch.cuda.OutOfMemoryError:
        error_print = f"torch.cuda.OutOfMemoryError: {model_name} and {Dataset_Name} ({df_Dataset_informs[df_Dataset_informs['Dataset Name']==Dataset_Name]['Input Shape'].item()})"
        Performances = [Experiments_Time, Train_date, iteration, Dataset_Name, 
                        f"'{int(ratio_combination[0]*100)}:{int(ratio_combination[1]*100)}:{int(ratio_combination[2]*100)}",
                        model_name, error_print, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                        0, 0, 0, 0, 0, transform_str, augmentation_str,
                        f"'{len(train_image_path_list)}/{len(validation_image_path_list)}/{len(test_image_path_list)}", FILENAME]
    
        # =========================== [SAFE-ROW-APPEND] ===========================
        _base_cols = [
            'Experiment Time','Train Time','Iteration','Dataset Name','Data Split','Model Name',
            'Val Loss','Test Loss','IoU','Dice','Precision','Recall','Total Params',
            'Train-Predction Time','Best Epoch','Time per Epoch',
            'Loss Function','LR','Batch size','#Epochs','Preprocessing','Augmentation',
            'Sample Size','DIR'
        ]
        _row = dict(zip(_base_cols, Performances))
        _extra_cols = [c for c in df.columns if c not in _base_cols]
        for c in _extra_cols:
            _row[c] = (ab_cfg.get(c, None) if 'ab_cfg' in locals() else None)
        _row_df = pd.DataFrame([[ _row.get(c, None) for c in df.columns ]], columns=df.columns)
        df = pd.concat([df, _row_df], ignore_index=True)
        # =======================================================================
    
        print('ERROR:', error_print)
    now = datetime.now()
    date=now.strftime("%y%m%d_%H%M%S")
    print('End',date)
    
    return df

def extend_to_full_batch(image_path_list, target_path_list, batch_size):
    # 원본 리스트의 길이를 batch_size로 나누어떨어지게 늘리는 함수
    num_samples = len(image_path_list)
    remainder = num_samples % batch_size

    if remainder != 0:
        # 필요한 만큼을 무작위로 추가해서 채운다
        extra_needed = batch_size - remainder
        available_indices = list(range(num_samples))  # 모든 인덱스에서 무작위로 추가할 인덱스 선택
        random_indices = random.choices(available_indices, k=extra_needed)  # 필요한 개수만큼 무작위로 선택

        for i in random_indices:
            image_path_list.append(image_path_list[i])
            target_path_list.append(target_path_list[i])
    return image_path_list,target_path_list

In [ ]:
now = datetime.now()
Experiments_Time=now.strftime("%y%m%d_%H%M%S")
print('Experiment Start Time:',Experiments_Time)
Metrics=['Experiment Time','Train Time', 'Iteration', 'Dataset Name', 'Data Split', 'Model Name', 'Val Loss', 'Test Loss', 'IoU', 'Dice',  'Precision', 'Recall', 'Total Params','Train-Predction Time','Best Epoch','Time per Epoch', 'Loss Function', 'LR', 'Batch size', '#Epochs', 'Preprocessing','Augmentation','Sample Size','DIR']
# [ABLATION] 'Model Name' 바로 뒤에 ablation 변수 컬럼 삽입
_insert_idx = Metrics.index('Model Name') + 1
Metrics = Metrics[:_insert_idx] + ablation_keys + Metrics[_insert_idx:]
df = pd.DataFrame(index=None, columns=Metrics)
output_root = f'{output_root}/output_{Experiments_Time}'
os.makedirs(output_root, exist_ok = True)
for ratio_combination in ratio_combinations:
    data_split_csv_list = [Dataset_root+'/'+f'(DataSplit)_Data_splits_{int(ratio_combination[0]*100)}_{int(ratio_combination[1]*100)}_{int(ratio_combination[2]*100)}_30'+'/'+f+f'_splits_{int(ratio_combination[0]*100)}_{int(ratio_combination[1]*100)}_{int(ratio_combination[2]*100)}_30.csv' for f in Dataset_Name_list]
    for iteration in range(iterations[0], iterations[1]+1):
    # for iteration in iterations:
        print(f'(Iter {iteration})')
        seed = iteration
        for augmentation_str in augmentation_strs:
            for j, Dataset_Name in enumerate(Dataset_Name_list):
                # if j+1<=19:
                #     continue
                print(f'Dataset: {Dataset_Name} ({j+1}/{len(Dataset_Name_list)})')
                control_random_seed(seed)

                Dataset_dir = Dataset_root+'/'+Dataset_Name
                data_split_csv = data_split_csv_list[j]

                Height, Width = parse_dimensions(df_Dataset_informs[df_Dataset_informs['Dataset Name']==Dataset_Name]['Image Shape (H,W)'].item())
                in_channels = int(df_Dataset_informs[df_Dataset_informs['Dataset Name']==Dataset_Name]['Inchannels'].item())
                number_of_classes = int(df_Dataset_informs[df_Dataset_informs['Dataset Name']==Dataset_Name]['Number of Classes'].item())
                BINARY_SEG = True if number_of_classes==2 else False
                exclude_background = EXCLUDE_BACKGROUND

                out_channels = 1 if BINARY_SEG else number_of_classes
                transform_str = df_Dataset_informs[df_Dataset_informs['Dataset Name']==Dataset_Name]['Transform'].item()
                transform_str = None if isinstance(transform_str, float) and np.isnan(transform_str) else transform_str
                print(f'Preprocessing: {transform_str}')
                # augmentation_str = df_Dataset_informs[df_Dataset_informs['Dataset Name']==Dataset_Name]['Augmentation'].item()
                # augmentation_str = None if isinstance(augmentation_str, float) and np.isnan(augmentation_str) else augmentation_str
                print(f'Augmentation: {augmentation_str}')

                evaluation_transform_str = df_Dataset_informs[df_Dataset_informs['Dataset Name']==Dataset_Name]['Evaluation'].item()
                evaluation_transform_str = None if isinstance(evaluation_transform_str, float) and np.isnan(evaluation_transform_str) else evaluation_transform_str
                print(f'Evaluation: {evaluation_transform_str}')

                (train_image_path_list, train_target_path_list,
                 validation_image_path_list, validation_target_path_list,
                 test_image_path_list, test_target_path_list) = create_dataset_lists(Dataset_dir, iteration, data_split_csv)

                print(f'train/val/test: {len(train_image_path_list)}/{len(validation_image_path_list)}/{len(test_image_path_list)}')

                if evaluation_transform_str and re.match(r'RC\((\d+),(\d+)\)', evaluation_transform_str):
                    validation_image_path_list = [path.replace('Originals','Originals_cropped') for path in validation_image_path_list for crop_count in range(1, 5)]
                    validation_target_path_list = [path.replace('Masks','Masks_cropped') for path in validation_target_path_list for crop_count in range(1, 5)]
                    test_image_path_list = [path.replace('Originals','Originals_cropped') for path in test_image_path_list for crop_count in range(1, 5)]
                    test_target_path_list = [path.replace('Masks','Masks_cropped') for path in test_target_path_list for crop_count in range(1, 5)]

                    # The lists now contain paths to the windowed images
                    print(f'train/cropped val/cropped test: {len(train_image_path_list)}/{len(validation_image_path_list)}/{len(test_image_path_list)}')

                # train_image_path_list = natsort.natsorted(train_image_path_list[:batch_size+1])
                # train_target_path_list = natsort.natsorted(train_target_path_list[:batch_size+1])
                # validation_image_path_list  = natsort.natsorted(validation_image_path_list[:batch_size+1])
                # validation_target_path_list = natsort.natsorted(validation_target_path_list[:batch_size+1])
                # test_image_path_list = natsort.natsorted(test_image_path_list[:batch_size+1])
                # test_target_path_list= natsort.natsorted(test_target_path_list[:batch_size+1])

                train_dataset = ImagesDataset(train_image_path_list, train_target_path_list, Height=Height, Width=Width, augmentation_str=augmentation_str, transform_str=transform_str)
                validation_dataset = ImagesDataset(validation_image_path_list, validation_target_path_list, Height=Height, Width=Width, augmentation_str=None, transform_str=transform_str)
                test_dataset = ImagesDataset(test_image_path_list, test_target_path_list, Height=Height, Width=Width, augmentation_str=None, transform_str=transform_str)
                train_loader = SegDataLoader(
                train_dataset, batch_size=batch_size,
                num_workers=4, pin_memory=True, shuffle=True, drop_last=drop_last, fill_last_batch=fill_last_batch,
                )
                validation_loader = SegDataLoader(
                    validation_dataset, batch_size=batch_size, 
                    num_workers=4, pin_memory=True,
                )
                test_loader = SegDataLoader(
                    test_dataset, batch_size=batch_size, 
                    num_workers=4, pin_memory=True,
                )

                for k, model_name in enumerate(model_names):
                    if train_mode=='inference' and ((iteration, Dataset_Name, model_name) not in iteration_dataset_model_tuples):
                        continue
                    if train_mode=='train_again' and ((iteration, Dataset_Name, model_name) not in iteration_dataset_model_tuples):
                        continue
                
                    print(f'{model_name} ({k+1}/{len(model_names)}) (Iter {iteration})', end=' ')
                    height, width, _ = map(int, df_Dataset_informs[df_Dataset_informs['Dataset Name']==Dataset_Name]['Input Shape'].item().strip('()').split(', '))
                    print(f'Dataset: {Dataset_Name} (Shape: ({height}, {width}, {in_channels})) ({j+1}/{len(Dataset_Name_list)})', end=' ')
                    print(f'Data Split: {int(ratio_combination[0]*100)}:{int(ratio_combination[1]*100)}:{int(ratio_combination[2]*100)}')
                
                    # [ABLATION] 여기서 조합별로 반복
                    for ab_cfg in ablation_grid:
                        ab_tag = ablation_to_tag(ab_cfg)
                        print(f"  -> Ablation: {ab_tag}")
                
                        # 출력 디렉토리에 ablation 태그 포함 (충돌 방지)
                        output_dir = (
                            output_root + f'/{model_name}_{Dataset_Name}'
                            + f'_Split_{int(ratio_combination[0]*100)}_{int(ratio_combination[1]*100)}_{int(ratio_combination[2]*100)}'
                            + f'_Iter_{iteration}_ABL_{ab_tag}'
                        )
                        copy_sourcefile(output_dir, src_dir='src')
                        control_random_seed(seed)
                
                        # 모델 생성: ablation 인자를 **우선** 전달, 실패 시 기존 방식 fallback
                        try:
                            if model_name in ['CSTF','DECSNet','CSTF_v2','SwinUNet','UTE_CrackNet']:
                                model = str_to_class(model_name)(in_channels, out_channels, resolution = (height, width), **ab_cfg)
                            else:
                                model = str_to_class(model_name)(in_channels, out_channels, **ab_cfg)
                        except TypeError:
                            # 기존 모델 시그니처가 ablation 인자를 받지 않는 경우
                            if model_name in ['CSTF','DECSNet','CSTF_v2','SwinUNet','UTE_CrackNet']:
                                model = str_to_class(model_name)(in_channels, out_channels, resolution = (height, width))
                            else:
                                model = str_to_class(model_name)(in_channels, out_channels)
                
                        device = torch.device("cuda:"+str(devices[0]))
                        if len(devices)>1 and (model_name not in ['Crackmer','DECSNet']):
                            model = torch.nn.DataParallel(model, device_ids = devices ).to(device)
                        else:
                            model = model.to(device)
                
                        # [ABLATION] Do_Experiment 전후로 df 길이 측정 → 새로 추가된 행에 ablation 값 채우기
                        _len_before = len(df)
                        df = Do_Experiment(
                            seed, model_name, model,
                            train_loader, validation_loader, test_loader,
                            optimizer, lr,  number_of_classes, epochs,
                            Metrics, df, device, None, train_mode
                        )
                        _len_after = len(df)
                
                        if _len_after > _len_before:
                            # 방금 추가된 구간(1개 이상일 수도 있음)에 대해 ablation 컬럼 값 채우기
                            for key in ablation_keys:
                                df.loc[_len_before:_len_after-1, key] = ab_cfg[key]
                
                        # CSV 저장
                        try:
                            df.to_csv(output_root+'/'+f'{Project_Name}_'+Experiments_Time+'.csv', index=False, header=True, encoding="cp949")
                        except:
                            now = datetime.now()
                            tmp_date=now.strftime("%y%m%d_%H%M%S")
                            df.to_csv(output_root+'/'+f'{Project_Name}_'+Experiments_Time+'_'+tmp_date+'_tmp'+'.csv', index=False, header=True, encoding="cp949")

import os
print('End')
os._exit(00) 

Experiment Start Time: 250822_100522
(Iter 4)
Dataset: CS01. Ceramic (1/5)
Preprocessing: None
Augmentation: FLIP(HV)=>ROT([90,180,270])
Evaluation: None
train/val/test: 60/20/20
UNet_MS_IG (1/1) (Iter 4) Dataset: CS01. Ceramic (Shape: (256, 256, 3)) (1/5) Data Split: 60:20:20
  -> Ablation: use_ms=True_use_gate=True
Training Start Time: 250822_100523
1EP(250822_100524): T_Loss: 0.553865 V_Loss: 0.491728 IoU: 0.0001 Best Epoch: 1 Loss: 0.491728
2EP(250822_100525): T_Loss: 0.473288 V_Loss: 0.420907 IoU: 0.2119 Best Epoch: 2 Loss: 0.420907
3EP(250822_100526): T_Loss: 0.424763 V_Loss: 0.399773 IoU: 0.2230 Best Epoch: 3 Loss: 0.399773
4EP(250822_100527): T_Loss: 0.409030 V_Loss: 0.393598 IoU: 0.2259 Best Epoch: 4 Loss: 0.393598
5EP(250822_100528): T_Loss: 0.396033 V_Loss: 0.368121 IoU: 0.2534 Best Epoch: 5 Loss: 0.368121
6EP(250822_100530): T_Loss: 0.386291 V_Loss: 0.342053 IoU: 0.2954 Best Epoch: 6 Loss: 0.342053
7EP(250822_100531): T_Loss: 0.377498 V_Loss: 0.354328 IoU: 0.2732 
8EP(25082

Test:   0%|          | 0/5 [00:00<?, ?batch/s]

Best Epoch: 67
Test(250822_100715): Loss: 0.311596 IoU: 0.3351 Dice: 0.4632 Precision: 0.4806 Recall: 0.5578
End 250822_100715
Dataset: CS02. CFD (2/5)
Preprocessing: HWR(384,512)=>P(384,512)
Augmentation: FLIP(HV)=>ROT([90,180,270])
Evaluation: None
train/val/test: 70/23/25
UNet_MS_IG (1/1) (Iter 4) Dataset: CS02. CFD (Shape: (384, 512, 3)) (2/5) Data Split: 60:20:20
  -> Ablation: use_ms=True_use_gate=True
Training Start Time: 250822_100715
1EP(250822_100718): T_Loss: 0.537227 V_Loss: 0.459985 IoU: 0.0000 Best Epoch: 1 Loss: 0.459985
2EP(250822_100721): T_Loss: 0.383950 V_Loss: 0.338947 IoU: 0.2950 Best Epoch: 2 Loss: 0.338947
3EP(250822_100724): T_Loss: 0.285798 V_Loss: 0.277043 IoU: 0.3873 Best Epoch: 3 Loss: 0.277043
4EP(250822_100728): T_Loss: 0.240694 V_Loss: 0.238773 IoU: 0.4356 Best Epoch: 4 Loss: 0.238773
5EP(250822_100731): T_Loss: 0.213014 V_Loss: 0.233763 IoU: 0.4386 Best Epoch: 5 Loss: 0.233763
6EP(250822_100734): T_Loss: 0.209834 V_Loss: 0.210346 IoU: 0.4793 Best Epoch: 

Test:   0%|          | 0/7 [00:00<?, ?batch/s]

Best Epoch: 30
Test(250822_101243): Loss: 0.187752 IoU: 0.5173 Dice: 0.6666 Precision: 0.6166 Recall: 0.7421
End 250822_101243
Dataset: CS03. DeepCrack237 (3/5)
Preprocessing: P(384,640)
Augmentation: FLIP(HV)=>ROT([90,180,270])
Evaluation: None
train/val/test: 142/47/48
UNet_MS_IG (1/1) (Iter 4) Dataset: CS03. DeepCrack237 (Shape: (384, 640, 3)) (3/5) Data Split: 60:20:20
  -> Ablation: use_ms=True_use_gate=True
Training Start Time: 250822_101243
1EP(250822_101251): T_Loss: 0.351615 V_Loss: 0.161044 IoU: 0.6593 Best Epoch: 1 Loss: 0.161044
2EP(250822_101259): T_Loss: 0.156923 V_Loss: 0.137029 IoU: 0.6666 Best Epoch: 2 Loss: 0.137029
3EP(250822_101307): T_Loss: 0.145522 V_Loss: 0.119399 IoU: 0.7028 Best Epoch: 3 Loss: 0.119399
4EP(250822_101315): T_Loss: 0.125518 V_Loss: 0.130577 IoU: 0.6772 
5EP(250822_101322): T_Loss: 0.120428 V_Loss: 0.126251 IoU: 0.6899 
6EP(250822_101330): T_Loss: 0.116210 V_Loss: 0.113991 IoU: 0.7085 Best Epoch: 6 Loss: 0.113991
7EP(250822_101338): T_Loss: 0.1099

...

Test:   0%|          | 0/12 [00:00<?, ?batch/s]

Best Epoch: 58
Test(250822_112022): Loss: 0.087310 IoU: 0.7827 Dice: 0.8678 Precision: 0.8687 Recall: 0.8835
End 250822_112022
Dataset: CS04. Masonry (4/5)
Preprocessing: R(256,256)
Augmentation: FLIP(HV)=>ROT([90,180,270])
Evaluation: None
train/val/test: 144/48/48
UNet_MS_IG (1/1) (Iter 5) Dataset: CS04. Masonry (Shape: (256, 256, 3)) (4/5) Data Split: 60:20:20
  -> Ablation: use_ms=True_use_gate=True
Training Start Time: 250822_112022
1EP(250822_112024): T_Loss: 0.346810 V_Loss: 0.248514 IoU: 0.5183 Best Epoch: 1 Loss: 0.248514
2EP(250822_112027): T_Loss: 0.216906 V_Loss: 0.191660 IoU: 0.5899 Best Epoch: 2 Loss: 0.191660
3EP(250822_112029): T_Loss: 0.199819 V_Loss: 0.183946 IoU: 0.5999 Best Epoch: 3 Loss: 0.183946
4EP(250822_112031): T_Loss: 0.185053 V_Loss: 0.174104 IoU: 0.6118 Best Epoch: 4 Loss: 0.174104
5EP(250822_112034): T_Loss: 0.174124 V_Loss: 0.199036 IoU: 0.5641 
6EP(250822_112036): T_Loss: 0.180586 V_Loss: 0.175186 IoU: 0.6122 
7EP(250822_112039): T_Loss: 0.175850 V_Loss:

Test:   0%|          | 0/12 [00:00<?, ?batch/s]

Best Epoch: 78
Test(250822_112424): Loss: 0.151291 IoU: 0.6547 Dice: 0.7716 Precision: 0.7820 Recall: 0.8018
End 250822_112424
Dataset: CS07. DeepCrack537 (5/5)
Preprocessing: P(384,640)
Augmentation: FLIP(HV)=>ROT([90,180,270])
Evaluation: None
train/val/test: 316/105/106
UNet_MS_IG (1/1) (Iter 5) Dataset: CS07. DeepCrack537 (Shape: (384, 640, 3)) (5/5) Data Split: 60:20:20
  -> Ablation: use_ms=True_use_gate=True
Training Start Time: 250822_112424
1EP(250822_112442): T_Loss: 0.275703 V_Loss: 0.168500 IoU: 0.6151 Best Epoch: 1 Loss: 0.168500
2EP(250822_112500): T_Loss: 0.150052 V_Loss: 0.151211 IoU: 0.6289 Best Epoch: 2 Loss: 0.151211
3EP(250822_112517): T_Loss: 0.134207 V_Loss: 0.139016 IoU: 0.6538 Best Epoch: 3 Loss: 0.139016
4EP(250822_112535): T_Loss: 0.130790 V_Loss: 0.129956 IoU: 0.6679 Best Epoch: 4 Loss: 0.129956
5EP(250822_112553): T_Loss: 0.122152 V_Loss: 0.124727 IoU: 0.6786 Best Epoch: 5 Loss: 0.124727
6EP(250822_112610): T_Loss: 0.116383 V_Loss: 0.121200 IoU: 0.6834 Best 

Test:   0%|          | 0/27 [00:00<?, ?batch/s]